# GTM Funnel Analysis - Marketing & Sales Performance

This notebook analyzes the GTM funnel connecting:
- Paid advertising performance (ad_spend)
- Web analytics (web_analytics)
- Sales opportunities (salesforce_opportunities)

**Goal**: Provide actionable insights for Marketing (Head of Growth) and Sales (Head of Sales Ops)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries loaded successfully!")

## 1. Load Source Data

In [ ]:
# Load CSV files
ad_spend = pd.read_csv('data/ad_spend.csv', parse_dates=['date'])
web_analytics = pd.read_csv('data/web_analytics.csv', parse_dates=['session_date'])
salesforce_opps = pd.read_csv('data/salesforce_opportunities.csv', parse_dates=['created_date'])

print(f"Ad Spend: {len(ad_spend):,} rows")
print(f"Web Analytics: {len(web_analytics):,} rows")
print(f"Salesforce Opportunities: {len(salesforce_opps):,} rows")

# Display sample data
print("\n=== Ad Spend Sample ===")
display(ad_spend.head(3))

print("\n=== Web Analytics Sample ===")
display(web_analytics.head(3))

print("\n=== Salesforce Opportunities Sample ===")
display(salesforce_opps.head(3))

## 2. Data Quality Overview

In [ ]:
# Check data quality and coverage
print("=== Ad Spend Data Quality ===")
print(f"Date range: {ad_spend['date'].min()} to {ad_spend['date'].max()}")
print(f"Channels: {ad_spend['channel'].unique()}")
print(f"Total spend: ${ad_spend['spend_usd'].sum():,.2f}")
print(f"Total clicks: {ad_spend['clicks'].sum():,}")
print(f"Total impressions: {ad_spend['impressions'].sum():,}")

print("\n=== Web Analytics Data Quality ===")
print(f"Date range: {web_analytics['session_date'].min()} to {web_analytics['session_date'].max()}")
print(f"Total sessions: {len(web_analytics):,}")
print(f"Sessions with UTM: {web_analytics['utm_source'].notna().sum():,} ({web_analytics['utm_source'].notna().sum()/len(web_analytics)*100:.1f}%)")
print(f"Sessions without UTM: {web_analytics['utm_source'].isna().sum():,} ({web_analytics['utm_source'].isna().sum()/len(web_analytics)*100:.1f}%)")
print(f"Total conversions: {web_analytics['conversions'].sum():,}")

print("\n=== Salesforce Opportunities Data Quality ===")
print(f"Date range: {salesforce_opps['created_date'].min()} to {salesforce_opps['created_date'].max()}")
print(f"Total opportunities: {len(salesforce_opps):,}")
print(f"\nBy Stage:")
print(salesforce_opps['stage'].value_counts())
print(f"\nBy Source:")
print(salesforce_opps['source'].value_counts())
print(f"\nTotal revenue (all): ${salesforce_opps['amount_usd'].sum():,.2f}")
print(f"Total revenue (Closed Won): ${salesforce_opps[salesforce_opps['stage']=='Closed Won']['amount_usd'].sum():,.2f}")

## 3. Build GTM Funnel by Channel

This is the core analysis connecting paid ads → web sessions → opportunities → closed-won revenue.

In [ ]:
# Aggregate ad spend by channel
ad_metrics = ad_spend.groupby('channel').agg({
    'spend_usd': 'sum',
    'clicks': 'sum',
    'impressions': 'sum'
}).reset_index()

# Calculate CTR and CPC
ad_metrics['ctr'] = ad_metrics['clicks'] / ad_metrics['impressions']
ad_metrics['cpc'] = ad_metrics['spend_usd'] / ad_metrics['clicks']

# Map UTM sources to channels for web analytics
channel_mapping = {
    'google': 'Google Ads',
    'linkedin': 'LinkedIn',
    'facebook': 'Meta',
    'twitter': 'Twitter'
}

web_analytics['channel'] = web_analytics['utm_source'].map(channel_mapping)

# Aggregate web analytics by channel
web_metrics = web_analytics[web_analytics['channel'].notna()].groupby('channel').agg({
    'session_id': 'count',
    'conversions': 'sum'
}).reset_index()
web_metrics.columns = ['channel', 'sessions', 'conversions']

# Aggregate Salesforce opportunities by source (channel)
opp_metrics = salesforce_opps.groupby('source').agg({
    'opportunity_id': 'count',
    'amount_usd': 'sum'
}).reset_index()
opp_metrics.columns = ['channel', 'total_opportunities', 'total_pipeline_value']

# Get closed-won metrics
closed_won = salesforce_opps[salesforce_opps['stage'] == 'Closed Won'].groupby('source').agg({
    'opportunity_id': 'count',
    'amount_usd': 'sum'
}).reset_index()
closed_won.columns = ['channel', 'closed_won_opps', 'closed_won_revenue']

# Merge all metrics into a single funnel dataframe
funnel = ad_metrics.copy()
funnel = funnel.merge(web_metrics, on='channel', how='left')
funnel = funnel.merge(opp_metrics, on='channel', how='left')
funnel = funnel.merge(closed_won, on='channel', how='left')

# Fill NaN values with 0
funnel = funnel.fillna(0)

# Calculate key GTM metrics
funnel['click_to_session_rate'] = funnel['sessions'] / funnel['clicks']
funnel['session_conversion_rate'] = funnel['conversions'] / funnel['sessions']
funnel['cost_per_session'] = funnel['spend_usd'] / funnel['sessions']
funnel['cost_per_conversion'] = funnel['spend_usd'] / funnel['conversions']
funnel['cost_per_opportunity'] = funnel['spend_usd'] / funnel['total_opportunities']
funnel['cac'] = funnel['spend_usd'] / funnel['closed_won_opps']
funnel['roi'] = (funnel['closed_won_revenue'] - funnel['spend_usd']) / funnel['spend_usd']
funnel['roas'] = funnel['closed_won_revenue'] / funnel['spend_usd']
funnel['opp_win_rate'] = funnel['closed_won_opps'] / funnel['total_opportunities']

# Replace inf values with NaN for display
funnel = funnel.replace([np.inf, -np.inf], np.nan)

print("GTM Funnel by Channel:")
display(funnel)

## 4. Visualization 1: Channel ROI & Efficiency Overview

**For**: Head of Growth  
**Question**: Which channels deliver the best return on ad spend?

In [ ]:
# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Channel Performance Overview - Marketing Efficiency', fontsize=16, fontweight='bold', y=1.00)

# Prepare data (remove channels with invalid metrics)
plot_data = funnel[funnel['roi'].notna()].copy()

# 1. ROI by Channel
ax1 = axes[0, 0]
bars1 = ax1.barh(plot_data['channel'], plot_data['roi'] * 100, color=['#2ecc71' if x > 0 else '#e74c3c' for x in plot_data['roi']])
ax1.set_xlabel('ROI (%)', fontweight='bold')
ax1.set_title('Return on Investment by Channel', fontweight='bold', fontsize=12)
ax1.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
ax1.grid(axis='x', alpha=0.3)
# Add value labels
for i, (idx, row) in enumerate(plot_data.iterrows()):
    ax1.text(row['roi'] * 100 + 5 if row['roi'] > 0 else row['roi'] * 100 - 5, 
             i, f"{row['roi']*100:.1f}%", 
             va='center', ha='left' if row['roi'] > 0 else 'right', fontweight='bold')

# 2. CAC by Channel
ax2 = axes[0, 1]
cac_data = plot_data[plot_data['cac'].notna()]
bars2 = ax2.barh(cac_data['channel'], cac_data['cac'], color='#3498db')
ax2.set_xlabel('Customer Acquisition Cost ($)', fontweight='bold')
ax2.set_title('CAC by Channel (Lower is Better)', fontweight='bold', fontsize=12)
ax2.grid(axis='x', alpha=0.3)
# Add value labels
for i, (idx, row) in enumerate(cac_data.iterrows()):
    ax2.text(row['cac'] + 50, i, f"${row['cac']:,.0f}", va='center', fontweight='bold')

# 3. Spend vs Revenue
ax3 = axes[1, 0]
x = np.arange(len(plot_data))
width = 0.35
bars3a = ax3.bar(x - width/2, plot_data['spend_usd'], width, label='Ad Spend', color='#e74c3c', alpha=0.8)
bars3b = ax3.bar(x + width/2, plot_data['closed_won_revenue'], width, label='Revenue', color='#2ecc71', alpha=0.8)
ax3.set_ylabel('Amount ($)', fontweight='bold')
ax3.set_title('Ad Spend vs. Closed-Won Revenue', fontweight='bold', fontsize=12)
ax3.set_xticks(x)
ax3.set_xticklabels(plot_data['channel'], rotation=45, ha='right')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)
# Format y-axis as currency
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# 4. ROAS by Channel
ax4 = axes[1, 1]
roas_data = plot_data[plot_data['roas'].notna()]
bars4 = ax4.barh(roas_data['channel'], roas_data['roas'], color='#9b59b6')
ax4.set_xlabel('Return on Ad Spend (X)', fontweight='bold')
ax4.set_title('ROAS by Channel (Revenue / Spend)', fontweight='bold', fontsize=12)
ax4.axvline(x=1, color='red', linestyle='--', linewidth=1, label='Break-even')
ax4.grid(axis='x', alpha=0.3)
ax4.legend()
# Add value labels
for i, (idx, row) in enumerate(roas_data.iterrows()):
    ax4.text(row['roas'] + 0.1, i, f"{row['roas']:.2f}x", va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('channel_roi_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved as: channel_roi_overview.png")

## 5. Visualization 2: Full Funnel Conversion Metrics

**For**: Head of Sales Ops & Head of Growth  
**Question**: Where are we losing prospects in the funnel?

In [ ]:
# Create funnel visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle('GTM Funnel Performance - Conversion Analysis', fontsize=16, fontweight='bold', y=0.98)

# Prepare funnel stages data
funnel_stages_data = []
for _, row in funnel.iterrows():
    if row['clicks'] > 0:  # Only include channels with data
        funnel_stages_data.append({
            'channel': row['channel'],
            'Impressions': row['impressions'],
            'Clicks': row['clicks'],
            'Sessions': row['sessions'],
            'Conversions': row['conversions'],
            'Opportunities': row['total_opportunities'],
            'Closed Won': row['closed_won_opps']
        })

funnel_df = pd.DataFrame(funnel_stages_data)

# 1. Funnel by Stage (Stacked)
ax1 = axes[0]
funnel_normalized = funnel_df.set_index('channel')[['Impressions', 'Clicks', 'Sessions', 'Conversions', 'Opportunities', 'Closed Won']]
funnel_normalized_pct = funnel_normalized.div(funnel_normalized['Impressions'], axis=0) * 100

funnel_normalized_pct.plot(kind='barh', stacked=False, ax=ax1, 
                            color=['#ecf0f1', '#3498db', '#9b59b6', '#e67e22', '#f39c12', '#2ecc71'],
                            alpha=0.8)
ax1.set_xlabel('% of Impressions', fontweight='bold')
ax1.set_title('Funnel Stage Volume (% of Impressions)', fontweight='bold', fontsize=12)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid(axis='x', alpha=0.3)
ax1.set_xlim(0, 100)

# 2. Conversion Rates Between Stages
ax2 = axes[1]
conversion_rates = funnel[['channel', 'ctr', 'click_to_session_rate', 'session_conversion_rate', 'opp_win_rate']].copy()
conversion_rates = conversion_rates[conversion_rates['ctr'].notna()]
conversion_rates_pct = conversion_rates.set_index('channel') * 100
conversion_rates_pct.columns = ['CTR\n(Click/Impr)', 'Click→Session\n(Sess/Click)', 'Session→Conv\n(Conv/Sess)', 'Opp→Won\n(Won/Opp)']

conversion_rates_pct.plot(kind='bar', ax=ax2, 
                          color=['#3498db', '#9b59b6', '#e67e22', '#2ecc71'],
                          alpha=0.8)
ax2.set_ylabel('Conversion Rate (%)', fontweight='bold')
ax2.set_title('Stage-to-Stage Conversion Rates', fontweight='bold', fontsize=12)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.legend(loc='upper left', fontsize=9)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('funnel_conversion_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved as: funnel_conversion_analysis.png")

## 6. Visualization 3: Key Metrics Summary Table

**For**: Executive Summary  
**Question**: Quick snapshot of channel performance

In [ ]:
# Create executive summary table
summary_table = funnel[[
    'channel', 'spend_usd', 'clicks', 'sessions', 'conversions', 
    'total_opportunities', 'closed_won_opps', 'closed_won_revenue',
    'cac', 'roi', 'roas'
]].copy()

# Format for display
summary_display = summary_table.copy()
summary_display['spend_usd'] = summary_display['spend_usd'].apply(lambda x: f'${x:,.0f}')
summary_display['clicks'] = summary_display['clicks'].apply(lambda x: f'{x:,.0f}')
summary_display['sessions'] = summary_display['sessions'].apply(lambda x: f'{x:,.0f}')
summary_display['conversions'] = summary_display['conversions'].apply(lambda x: f'{x:,.0f}')
summary_display['total_opportunities'] = summary_display['total_opportunities'].apply(lambda x: f'{x:,.0f}')
summary_display['closed_won_opps'] = summary_display['closed_won_opps'].apply(lambda x: f'{x:,.0f}')
summary_display['closed_won_revenue'] = summary_display['closed_won_revenue'].apply(lambda x: f'${x:,.0f}')
summary_display['cac'] = summary_display['cac'].apply(lambda x: f'${x:,.0f}' if pd.notna(x) else 'N/A')
summary_display['roi'] = summary_display['roi'].apply(lambda x: f'{x*100:.1f}%' if pd.notna(x) else 'N/A')
summary_display['roas'] = summary_display['roas'].apply(lambda x: f'{x:.2f}x' if pd.notna(x) else 'N/A')

# Rename columns for clarity
summary_display.columns = [
    'Channel', 'Ad Spend', 'Clicks', 'Sessions', 'Conversions',
    'Opportunities', 'Closed Won', 'Revenue', 'CAC', 'ROI', 'ROAS'
]

# Create styled table visualization
fig, ax = plt.subplots(figsize=(18, 6))
ax.axis('tight')
ax.axis('off')

# Create table
table = ax.table(cellText=summary_display.values,
                 colLabels=summary_display.columns,
                 cellLoc='center',
                 loc='center',
                 colWidths=[0.08, 0.08, 0.08, 0.08, 0.08, 0.10, 0.08, 0.10, 0.08, 0.08, 0.08])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# Style header
for i in range(len(summary_display.columns)):
    table[(0, i)].set_facecolor('#2c3e50')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Style rows with alternating colors
for i in range(1, len(summary_display) + 1):
    for j in range(len(summary_display.columns)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#ecf0f1')
        else:
            table[(i, j)].set_facecolor('white')

plt.title('GTM Channel Performance Summary', fontsize=14, fontweight='bold', pad=20)
plt.savefig('channel_summary_table.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved as: channel_summary_table.png")

# Also display as pandas styled dataframe
print("\n=== Channel Performance Summary ===")
display(summary_display)

## 7. Time-Based Trends (Bonus Analysis)

In [ ]:
# Analyze trends over time
# Group ad spend by week and channel
ad_spend['week'] = ad_spend['date'].dt.to_period('W').dt.start_time
weekly_spend = ad_spend.groupby(['week', 'channel'])['spend_usd'].sum().reset_index()

# Group web analytics by week
web_analytics['week'] = web_analytics['session_date'].dt.to_period('W').dt.start_time
weekly_sessions = web_analytics[web_analytics['channel'].notna()].groupby(['week', 'channel']).agg({
    'session_id': 'count',
    'conversions': 'sum'
}).reset_index()
weekly_sessions.columns = ['week', 'channel', 'sessions', 'conversions']

# Group opportunities by week
salesforce_opps['week'] = salesforce_opps['created_date'].dt.to_period('W').dt.start_time
weekly_opps = salesforce_opps.groupby(['week', 'source']).agg({
    'opportunity_id': 'count',
    'amount_usd': 'sum'
}).reset_index()
weekly_opps.columns = ['week', 'channel', 'opportunities', 'pipeline_value']

# Create weekly trend visualization
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Weekly Trends by Channel', fontsize=16, fontweight='bold')

# 1. Weekly Spend
ax1 = axes[0, 0]
for channel in weekly_spend['channel'].unique():
    data = weekly_spend[weekly_spend['channel'] == channel]
    ax1.plot(data['week'], data['spend_usd'], marker='o', label=channel, linewidth=2)
ax1.set_ylabel('Ad Spend ($)', fontweight='bold')
ax1.set_title('Weekly Ad Spend Trends', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 2. Weekly Sessions
ax2 = axes[0, 1]
for channel in weekly_sessions['channel'].unique():
    data = weekly_sessions[weekly_sessions['channel'] == channel]
    ax2.plot(data['week'], data['sessions'], marker='o', label=channel, linewidth=2)
ax2.set_ylabel('Sessions', fontweight='bold')
ax2.set_title('Weekly Session Trends', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# 3. Weekly Conversions
ax3 = axes[1, 0]
for channel in weekly_sessions['channel'].unique():
    data = weekly_sessions[weekly_sessions['channel'] == channel]
    ax3.plot(data['week'], data['conversions'], marker='o', label=channel, linewidth=2)
ax3.set_ylabel('Conversions', fontweight='bold')
ax3.set_title('Weekly Conversion Trends', fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# 4. Weekly Opportunities
ax4 = axes[1, 1]
for channel in weekly_opps['channel'].unique():
    data = weekly_opps[weekly_opps['channel'] == channel]
    ax4.plot(data['week'], data['opportunities'], marker='o', label=channel, linewidth=2)
ax4.set_ylabel('Opportunities', fontweight='bold')
ax4.set_title('Weekly Opportunity Trends', fontweight='bold')
ax4.legend()
ax4.grid(alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('weekly_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved as: weekly_trends.png")

## 8. Key Insights & Recommendations

Based on the analysis above, here are the key findings:

In [ ]:
# Generate automated insights
print("=" * 80)
print("KEY INSIGHTS & RECOMMENDATIONS")
print("=" * 80)

# Sort by ROI
funnel_ranked = funnel[funnel['roi'].notna()].sort_values('roi', ascending=False)

print("\n📊 CHANNEL ROI RANKING (Best to Worst):")
for i, (_, row) in enumerate(funnel_ranked.iterrows(), 1):
    print(f"  {i}. {row['channel']:12s} - ROI: {row['roi']*100:6.1f}% | ROAS: {row['roas']:5.2f}x | CAC: ${row['cac']:,.0f}")

# Find best channel
best_channel = funnel_ranked.iloc[0]
print(f"\n✅ BEST PERFORMER: {best_channel['channel']}")
print(f"   - Highest ROI at {best_channel['roi']*100:.1f}%")
print(f"   - ROAS: {best_channel['roas']:.2f}x (every $1 spent returns ${best_channel['roas']:.2f})")
print(f"   - CAC: ${best_channel['cac']:,.0f}")

# Find worst channel
worst_channel = funnel_ranked.iloc[-1]
print(f"\n⚠️  NEEDS OPTIMIZATION: {worst_channel['channel']}")
print(f"   - ROI: {worst_channel['roi']*100:.1f}%")
print(f"   - ROAS: {worst_channel['roas']:.2f}x")
print(f"   - CAC: ${worst_channel['cac']:,.0f}")

# Calculate overall metrics
total_spend = funnel['spend_usd'].sum()
total_revenue = funnel['closed_won_revenue'].sum()
overall_roi = (total_revenue - total_spend) / total_spend

print(f"\n💰 OVERALL GTM PERFORMANCE:")
print(f"   - Total Ad Spend: ${total_spend:,.0f}")
print(f"   - Total Revenue (Closed Won): ${total_revenue:,.0f}")
print(f"   - Blended ROI: {overall_roi*100:.1f}%")
print(f"   - Total Opportunities: {funnel['total_opportunities'].sum():.0f}")
print(f"   - Total Closed Won: {funnel['closed_won_opps'].sum():.0f}")

print("\n" + "=" * 80)
print("RECOMMENDATIONS FOR HEAD OF GROWTH & HEAD OF SALES OPS")
print("=" * 80)

print(f"\n1. INCREASE INVESTMENT in {best_channel['channel']}")
print(f"   → Highest ROI channel with proven performance")
print(f"   → Consider increasing budget by 20-30% to scale wins")

print(f"\n2. OPTIMIZE OR REDUCE SPEND on {worst_channel['channel']}")
print(f"   → Lower ROI suggests need for creative/targeting optimization")
print(f"   → Test new messaging or reallocate budget to higher-performing channels")

print(f"\n3. FOCUS ON CONVERSION OPTIMIZATION")
avg_session_conv_rate = funnel['session_conversion_rate'].mean() * 100
print(f"   → Average session conversion rate: {avg_session_conv_rate:.2f}%")
print(f"   → Improving landing pages and CTAs could boost conversions 10-20%")

print(f"\n4. IMPROVE ATTRIBUTION TRACKING")
unattributed_sessions = web_analytics['utm_source'].isna().sum()
unattributed_pct = unattributed_sessions / len(web_analytics) * 100
print(f"   → {unattributed_pct:.1f}% of sessions lack UTM parameters")
print(f"   → Implement UTM tagging standards across all campaigns")

print("\n" + "=" * 80)

## 9. Data Quality & Assumptions

**Attribution Methodology:**
- **Ad Spend → Web Sessions**: Joined on UTM source and campaign parameters
- **Web Sessions → Opportunities**: Mapped via channel/source (no session-level tracking)
- **Attribution Model**: Rule-based, last-touch implied via source field
- **Time Window**: No explicit time-window attribution (opportunity created_date may lag campaign activity)

**Known Limitations:**
1. ~20-30% of web sessions lack UTM parameters (direct/organic traffic)
2. No multi-touch attribution model (first-touch, linear, time-decay)
3. Time lag between ad impression → session → opportunity not explicitly modeled
4. Source field in Salesforce may be manually entered (data quality risk)

**Recommendations for Next Steps:**
- Implement time-window attribution logic (e.g., opportunities within 30 days of session)
- Build multi-touch attribution models for more accurate credit assignment
- Add data quality tests for UTM parameter consistency
- Incorporate customer journey data for better session-to-opportunity linkage

## 10. Export Results for Stakeholder Presentation

In [ ]:
# Export funnel data to CSV for further analysis
funnel.to_csv('gtm_funnel_results.csv', index=False)
print("✅ Funnel results exported to: gtm_funnel_results.csv")

# Export summary table
summary_display.to_csv('channel_summary.csv', index=False)
print("✅ Summary table exported to: channel_summary.csv")

print("\n📊 All visualizations saved:")
print("   1. channel_roi_overview.png")
print("   2. funnel_conversion_analysis.png")
print("   3. channel_summary_table.png")
print("   4. weekly_trends.png")

print("\n✅ Analysis complete! Ready for stakeholder presentation.")